#### installs

In [ ]:
!pip install git+https://github.com/openai/whisper.git 

In [5]:
!sudo apt install ffmpeg

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libass9 libasyncns0 libavc1394-0 libavdevice58 libavfilter7 libavresample4
  libbs2b0 libcaca0 libcdio-cdda2 libcdio-paranoia2 libcdio18 libdc1394-22
  libfftw3-double3 libflac8 libflite1 libiec61883-0 libjack-jackd2-0
  liblilv-0-0 libmysofa1 libnorm1 libopenal-data libopenal1 libpgm-5.2-0
  libpostproc55 libpulse0 libraw1394-11 librubberband2 libsamplerate0
  libsdl2-2.0-0 libserd-0-0 libslang2 libsndfile1 libsndio7.0 libsodium23
  libsord-0-0 libsratom-0-0 libusb-1.0-0 libvidstab1.1 libxss1 libxv1 libzmq5
Suggested packages:
  ffmpeg-doc libfftw3-bin libfftw3-dev jackd2 libportaudio2 pulseaudio
  libraw1394-doc serdi sndiod sordi
The following NEW packages will be installed:
  ffmpeg libass9 libasyncns0 libavc1394-0 libavdevice58 libavfilter7
  libavresample4 libbs2b0 libcaca0 libcdio-cdda2 libcdio-paranoia2 libcdio18
  libdc1394-22 l

In [6]:
pip install setuptools-rust

Note: you may need to restart the kernel to use updated packages.


#### model

In [7]:
import whisper

model = whisper.load_model('base')
result = model.transcribe('input_audio.wav')
print(result['text'])

100%|███████████████████████████████████████| 139M/139M [00:02<00:00, 64.6MiB/s]
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


 Say I'm running a coin flip experiment, and I want to find out how likely each alchemy's heads or tails. So I public coin once, twice, a hundred times, and once I've repeated that experiment enough times, I see that about 50% of my flips or heads and 50% are tails. Now, that's not a particularly interesting result. You probably could have told me that's what would happen at the beginning. But what if the experiment I want to repeat is much bigger? Instead of physically performing the experiment, we can simulate it with code. For example, maybe I want to simulate a car crash to predict the risk of injury to the passengers. Or I want to simulate a forest fire to predict how far it'll spread. Or I want to simulate crop growth so I can predict yields and decide what to plant. These are all things that would be far too costly, too devastating, or take far too long to repeat in the real world. But if we build a computer simulation, we can repeat the experiment as many times as we want for f

In [21]:
import whisper
model = whisper.load_model('base')

# using ffmpeg to load the audio.
# returns dim audio array.
audio = whisper.load_audio('input_audio.wav')
print('original audio shape', audio.shape)

# checks for size of the audio and trim it or padded with zeros it if it's more that 30 sec length
# if you pass more than 30 sec audio you will lose the after 30 sec data.
# so it may be necessary to chunk your data to 30sec before passing it to the model.
audio = whisper.pad_or_trim(audio)
print('trimed (30sec) audio shape', audio.shape)

original audio shape (2479200,)
trimed (30sec) audio shape (480000,)


In [ ]:
# index select technique
import torch
x = torch.randn(100)
print(x[:2])
x.index_select(-1, torch.arange(2))

In [34]:
# padding technique
import torch
import torch.nn.functional as F
x = torch.randn(5)
pad_width = [(0, 0)] * x.ndim
pad_width[-1] = (0, 10 - x.shape[-1])
x = F.pad(x, [pad for sz in pad_width[::-1] for pad in sz ])
[pad for sz in pad_width[::-1] for pad in sz ]
pad_width[::-1]

[(0, 5)]

In [15]:
whisper.pad_or_trim??

Signature: whisper.pad_or_trim(array, length: int = 480000, *, axis: int = -1)
Source:   
def pad_or_trim(array, length: int = N_SAMPLES, *, axis: int = -1):
    """
    Pad or trim the audio array to N_SAMPLES, as expected by the encoder.
    """
    if torch.is_tensor(array):
        if array.shape[axis] > length:
            array = array.index_select(
                dim=axis, index=torch.arange(length, device=array.device)
            )

        if array.shape[axis] < length:
            pad_widths = [(0, 0)] * array.ndim
            pad_widths[axis] = (0, length - array.shape[axis])
            array = F.pad(array, [pad for sizes in pad_widths[::-1] for pad in sizes])
    else:
        if array.shape[axis] > length:
            array = array.take(indices=range(length), axis=axis)

        if array.shape[axis] < length:
            pad_widths = [(0, 0)] * array.ndim
            pad_widths[axis] = (0, length - array.shape[axis])
            array = np.pad(array, pad_widths)

    re

In [ ]:
import whisper

model = whisper.load_model("base")

# load audio and pad/trim it to fit 30 seconds
audio = whisper.load_audio("audio.mp3")
audio = whisper.pad_or_trim(audio)

# make log-Mel spectrogram and move to the same device as the model
mel = whisper.log_mel_spectrogram(audio).to(model.device)

# detect the spoken language
_, probs = model.detect_language(mel)
print(f"Detected language: {max(probs, key=probs.get)}")

# decode the audio
options = whisper.DecodingOptions()
result = whisper.decode(model, mel, options)

# print the recognized text
print(result.text)